In [ ]:
from sys import path
from os import getcwd

path.insert(getcwd() + '/modules/') # adding modules folder to path

from modules.DataMod import DataSet
from modules.TrainingManager import KerasTrainingManager
from modules.TrainingFunctions import generator_training
from modules.misc import LSSIM, ssim_metric, psnrb_metric

from tensorflow.keras.optimizers import Adam


environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
# creates the datasets
tinyDataSet = DataSet.load_rafael_tinyImagenet_64x64_noise_data()
cifarDataSet = DataSet.load_rafael_cifar_10_noise_data()
cifarAndTinyDataSet = DataSet.concatenateDataSets(cifarDataSet, tinyDataSet)

## Training Models

In [ ]:
# to do: paralelize the training (does it's necessary?)

# uNet

# trainiing unet only with tinyImagenet dataset
uNet_tiny_Manager = KerasTrainingManager(
    "Unet2.3-64x64.json",
    optimizer = Adam,
    optimizer_kwargs = {'learning_rate' : 0.001, 'beta_1' : 0.9, 'beta_2' : 0.999, 'epsilon' : 1e-7, 'amsgrad' : False},
    
    fit_kwargs = {'batch_size' : 20, 'epochs' : 15, 'verbose':1, 'validation_split':0, 'shuffle':True, 
    'class_weight':None, 'sample_weight':None, 'steps_per_epoch':None, 'validation_steps':None, 
    'validation_batch_size':None, 'validation_freq':1, 'max_queue_size':10, 'workers':1, 'use_multiprocessing':False},

    metrics = [ssim_metric, psnrb_metric, LSSIM],

    training_function = generator_training,

    dataset = tinyDataSet
)

# trainiing unet only with cifar dataset
uNet_cifar_Manager = KerasTrainingManager(
    "Unet2.3-64x64.json",
    optimizer = Adam,
    optimizer_kwargs = {'learning_rate' : 0.001, 'beta_1' : 0.9, 'beta_2' : 0.999, 'epsilon' : 1e-7, 'amsgrad' : False},
    
    fit_kwargs = {'batch_size' : 20, 'epochs' : 15, 'verbose':1, 'validation_split':0, 'shuffle':True, 
    'class_weight':None, 'sample_weight':None, 'steps_per_epoch':None, 'validation_steps':None, 
    'validation_batch_size':None, 'validation_freq':1, 'max_queue_size':10, 'workers':1, 'use_multiprocessing':False},

    metrics = [ssim_metric, psnrb_metric, LSSIM], # not sure about it

    training_function = generator_training,

    dataset = cifarDataSet
)

# training unet with both datasets concatenated
uNet_cifarAndTiny_Manager = KerasTrainingManager(
    "Unet2.3-64x64.json",
    optimizer = Adam,
    optimizer_kwargs = {'learning_rate' : 0.001, 'beta_1' : 0.9, 'beta_2' : 0.999, 'epsilon' : 1e-7, 'amsgrad' : False},
    
    fit_kwargs = {'batch_size' : 20, 'epochs' : 15, 'verbose':1, 'validation_split':0, 'shuffle':True, 
    'class_weight':None, 'sample_weight':None, 'steps_per_epoch':None, 'validation_steps':None, 
    'validation_batch_size':None, 'validation_freq':1, 'max_queue_size':10, 'workers':1, 'use_multiprocessing':False},

    metrics = [ssim_metric, psnrb_metric, LSSIM],

    training_function = generator_training,

    dataset = cifarAndTinyDataSet
)


In [ ]:

# autoencoder

# training autoencoder only with tinyImagenet dataset
autoencoder_tiny_Manager = KerasTrainingManager(
    "AutoEncoder-2.3-64x64",
    optimizer = Adam,
    optimizer_kwargs = {'learning_rate' : 0.001, 'beta_1' : 0.9, 'beta_2' : 0.999, 'epsilon' : 1e-7, 'amsgrad' : False},
    
    fit_kwargs = {'batch_size' : 20, 'epochs' : 15, 'verbose':1, 'validation_split':0, 'shuffle':True, 
    'class_weight':None, 'sample_weight':None, 'steps_per_epoch':None, 'validation_steps':None, 
    'validation_batch_size':None, 'validation_freq':1, 'max_queue_size':10, 'workers':1, 'use_multiprocessing':False},

    metrics = [ssim_metric, psnrb_metric, LSSIM],

    training_function = generator_training,

    dataset = tinyDataSet
)

# training autoencoder only with cifar dataset
autoencoder_cifar_Manager = KerasTrainingManager(
    "AutoEncoder-2.3-64x64",
    optimizer = Adam,
    optimizer_kwargs = {'learning_rate' : 0.001, 'beta_1' : 0.9, 'beta_2' : 0.999, 'epsilon' : 1e-7, 'amsgrad' : False},
    
    fit_kwargs = {'batch_size' : 20, 'epochs' : 15, 'verbose':1, 'validation_split':0, 'shuffle':True, 
    'class_weight':None, 'sample_weight':None, 'steps_per_epoch':None, 'validation_steps':None, 
    'validation_batch_size':None, 'validation_freq':1, 'max_queue_size':10, 'workers':1, 'use_multiprocessing':False},

    metrics = [ssim_metric, psnrb_metric, LSSIM],

    training_function = generator_training,

    dataset = cifarDataSet
)

# training autoencoder with both datasets concatenated
autoencoder_cifarAndTiny_Manager = KerasTrainingManager(
    "AutoEncoder-2.3-64x64",
    optimizer = Adam,
    optimizer_kwargs = {'learning_rate' : 0.001, 'beta_1' : 0.9, 'beta_2' : 0.999, 'epsilon' : 1e-7, 'amsgrad' : False},
    
    fit_kwargs = {'batch_size' : 20, 'epochs' : 15, 'verbose':1, 'validation_split':0, 'shuffle':True, 
    'class_weight':None, 'sample_weight':None, 'steps_per_epoch':None, 'validation_steps':None, 
    'validation_batch_size':None, 'validation_freq':1, 'max_queue_size':10, 'workers':1, 'use_multiprocessing':False},

    metrics = [ssim_metric, psnrb_metric, LSSIM],

    training_function = generator_training,

    dataset = cifarAndTinyDataSet
)


In [ ]:
# residual autoencoder
residual_autoencoder_tiny_Manager = KerasTrainingManager(
    "ResidualAutoEncoder-0.1-64x64",
    optimizer = Adam,
    optimizer_kwargs = {'learning_rate' : 0.001, 'beta_1' : 0.9, 'beta_2' : 0.999, 'epsilon' : 1e-7, 'amsgrad' : False},
    
    fit_kwargs = {'max_val': 255, 'filter_size': 9, 'filter_sigma': 1.5, 'k1':0.01, 'k2': 0.03, 'batch_size' : 20, 'epochs' : 25, 'verbose':1, 'validation_split':0, 'shuffle':True, 
    'class_weight':None, 'sample_weight':None, 'steps_per_epoch':None, 'validation_steps':None, 
    'validation_batch_size':None, 'validation_freq':1, 'max_queue_size':10, 'workers':1, 'use_multiprocessing':False},

    metrics = [ssim_metric, psnrb_metric, LSSIM], # not sure about it

    training_function = generator_training,

    dataset = tinyDataSet
)
residual_autoencoder_cifar_Manager = KerasTrainingManager(
    "ResidualAutoEncoder-0.1-64x64",
    optimizer = Adam,
    optimizer_kwargs = {'learning_rate' : 0.001, 'beta_1' : 0.9, 'beta_2' : 0.999, 'epsilon' : 1e-7, 'amsgrad' : False},
    
    fit_kwargs = {'max_val': 255, 'filter_size': 9, 'filter_sigma': 1.5, 'k1':0.01, 'k2': 0.03, 'batch_size' : 20, 'epochs' : 25, 'verbose':1, 'validation_split':0, 'shuffle':True, 
    'class_weight':None, 'sample_weight':None, 'steps_per_epoch':None, 'validation_steps':None, 
    'validation_batch_size':None, 'validation_freq':1, 'max_queue_size':10, 'workers':1, 'use_multiprocessing':False},

    metrics = [ssim_metric, psnrb_metric, LSSIM], # not sure about it

    training_function = generator_training,

    dataset = cifarDataSet
)

residual_autoencoder_cifarAndTiny_Manager = KerasTrainingManager(
    "ResidualAutoEncoder-0.1-64x64",
    optimizer = Adam,
    optimizer_kwargs = {'learning_rate' : 0.001, 'beta_1' : 0.9, 'beta_2' : 0.999, 'epsilon' : 1e-7, 'amsgrad' : False},
    
    fit_kwargs = {'max_val': 255, 'filter_size': 9, 'filter_sigma': 1.5, 'k1':0.01, 'k2': 0.03, 'batch_size' : 20, 'epochs' : 25, 'verbose':1, 'validation_split':0, 'shuffle':True, 
    'class_weight':None, 'sample_weight':None, 'steps_per_epoch':None, 'validation_steps':None, 
    'validation_batch_size':None, 'validation_freq':1, 'max_queue_size':10, 'workers':1, 'use_multiprocessing':False},

    metrics = [ssim_metric, psnrb_metric, LSSIM], # not sure about it

    training_function = generator_training,

    dataset = cifarAndTinyDataSet
)

In [ ]:
# starting models training 
uNet_tiny_Manager.start_training()
uNet_cifar_Manager.start_training()
uNet_cifarAndTiny_Manager.start_training()

autoencoder_tiny_Manager.start_training()
autoencoder_cifar_Manager.start_training()
autoencoder_cifarAndTiny_Manager.start_training()

residual_autoencoder_tiny_Manager.start_training()
residual_autoencoder_cifar_Manager.start_training()
residual_autoencoder_cifarAndTiny_Manager.start_training()